In [ ]:
import pymysql
import pandas as pd
import boto3
from datetime import datetime

In [ ]:
def main():
    host = "hyundb.cqgrmpcsglic.ap-northeast-2.rds.amazonaws.com"
    port = 3306
    username = 'hyunjilee'
    database = 'production'
    pw = ''

    conn = pymysql.connect(host=host, user=username, passwd=pw,
                          db=database, port=port,
                          use_unicode=True, charset='utf8')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM artists')
    a_data = cursor.fetchall()
    col = [d[0] for d in cursor.description]
    data = pd.DataFrame(a_data, columns=col)
    data.to_parquet('artists.parquet', engine='pyarrow',
                compression='snappy')
    dt = datetime.utcnow().strftime('%Y-%m-%d')
    s3 = boto3.resource('s3', region_name='ap-northeast-2')
    ob = s3.Object('hyun-artist', 'artists/dt={}/artists.parquet'.format(dt))
    data = open('artists.parquet', 'rb')
    ob.put(Body=data)

In [ ]:
if __name__ == '__main__':
    main()